<a href="https://colab.research.google.com/github/KendyHuynh/Bigdata_project/blob/main/projectbigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Cài đặt PySpark
!pip install pyspark

# Khởi tạo SparkSession và đọc dữ liệu từ tập tin log:

Sử dụng SparkSession để khởi tạo một phiên làm việc Spark.

Sử dụng phương thức read.text() để đọc dữ liệu từ tập tin log vào một DataFrame.

In [ ]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("readdatalog") \
    .getOrCreate()

# Đường dẫn đến tập tin log
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Đọc dữ liệu log từ tập tin
log_data = spark.read.text(log_file_path)

# Hiển thị schema của DataFrame
log_data.printSchema()

# Hiển thị 20 dòng đầu tiên của DataFrame
log_data.show(20, truncate=False)

# Đóng SparkSession
spark.stop()



root
 |-- value: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                 

**Nhìn vào dữ liệu log đọc được ta thấy rằng có các thông tin như sau:**

1. IP_Address (31.56.96.51)

2. TimeStamp ([22/Jan/2019...])

3. Request_HTTP ("GET /image/60844/productModel/...")

4. Status_Code (200)

5. Response_Size (5667)

6. Referrer ("https://www.zanbil.ir/...")

7. User_Agent ("Mozilla/5.0 (Linux; Android 6.0...")

8. Additional_Info ("-")

#Phân tích log và tạo DataFrame mới:

Sử dụng biểu thức chính quy để trích xuất thông tin từ dòng log, bao gồm địa chỉ IP, thời gian, yêu cầu, mã trạng thái, kích thước phản hồi, referrer, user agent và thông tin bổ sung.

Sử dụng regexp_extract() để áp dụng biểu thức chính quy và tạo các cột mới trong DataFrame.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

# Khởi tạo SparkSessi on
spark = SparkSession.builder \
    .appName("LogAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào DataFrame và phân tích các trường thông tin
parsed_logs_df = spark.read.text(log_file_path).select(
    regexp_extract('value', log_regex, 1).alias('ip_address'),
    regexp_extract('value', log_regex, 2).alias('timestamp'),
    regexp_extract('value', log_regex, 3).alias('request'),
    regexp_extract('value', log_regex, 4).alias('status_code'),
    regexp_extract('value', log_regex, 5).alias('response_size'),
    regexp_extract('value', log_regex, 6).alias('referrer'),
    regexp_extract('value', log_regex, 7).alias('user_agent'),
    regexp_extract('value', log_regex, 8).alias('additional_info')
)

# Hiển thị DataFrame sau khi đã phân tích các trường thông tin
parsed_logs_df.show(50,truncate=False)

+-------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|ip_address   |timestamp                 |request                                                                                                                                                                                                      |status_code|response_size|referrer              

#Phân tích dữ liệu và thống kê:


1. Thống kê số lượng yêu cầu theo địa chỉ IP:

In [ ]:
ip_counts_rdd = parsed_logs_df.rdd.map(lambda x: (x['ip_address'], 1)).reduceByKey(lambda a, b: a + b)
ip_counts = spark.createDataFrame(ip_counts_rdd, ['ip_address', 'count'])
print("Số lượng yêu cầu theo địa chỉ IP:")
ip_counts.show(truncate=False)

+--------------+-----+
|ip_address    |count|
+--------------+-----+
|5.62.206.249  |3    |
|54.36.148.197 |165  |
|63.143.42.246 |4500 |
|54.36.148.21  |161  |
|5.123.139.174 |4    |
|54.36.148.95  |171  |
|67.195.177.230|2    |
|54.36.149.61  |158  |
|54.36.148.144 |153  |
|5.113.178.253 |31   |
|54.36.149.67  |178  |
|54.36.148.140 |168  |
|207.154.254.69|7    |
|37.129.110.227|3    |
|213.174.147.83|21   |
|5.121.43.23   |166  |
|204.18.139.145|5    |
|89.34.54.44   |1    |
|123.125.71.31 |3    |
|54.36.148.231 |154  |
+--------------+-----+
only showing top 20 rows



2. Thống kê số lượng yêu cầu theo mã trạng thái:

In [ ]:
status_code_counts_rdd = parsed_logs_df.rdd.map(lambda x: (x['status_code'], 1)).reduceByKey(lambda a, b: a + b)
status_code_counts = spark.createDataFrame(status_code_counts_rdd, ['status_code', 'count'])
print("Số lượng yêu cầu theo mã trạng thái:")
status_code_counts.show(truncate=False)

Số lượng yêu cầu theo mã trạng thái:
+-----------+-------+
|status_code|count  |
+-----------+-------+
|504        |103    |
|           |2      |
|301        |67551  |
|502        |798    |
|200        |9579825|
|499        |50852  |
|302        |199835 |
|414        |17     |
|500        |14266  |
|400        |586    |
|408        |112    |
|401        |323    |
|405        |6      |
|304        |340228 |
|403        |5634   |
|206        |3      |
|404        |105011 |
+-----------+-------+



**Nhận xét:**

***1. Số lượng request theo phương thức:***
- **GET**: Số lượng request sử dụng phương thức GET là **10,190,003**.
- **POST**: Số lượng request sử dụng phương thức POST là **139,155**.
- **HEAD**: Số lượng request sử dụng phương thức HEAD là **34,501**.
- **OPTIONS**: Số lượng request sử dụng phương thức OPTIONS là **1,424**.
- **CONNECT**: Số lượng request sử dụng phương thức CONNECT là **10**.
- Các phương thức còn lại như **PROPFIND**, **PUT**, **TRACE**, và **một số phương thức không hợp lệ** như "*\x17\x03\x03\x01\xBB\x00\x00\x00\x00\x00\x00\x00\x1F\x8A\x03\x9F*" có **số lần xuất hiện nhỏ và không đáng kể**.

***2. Số lượng lỗi:***
- **404 Not Found:** Số lượng lỗi 404 (Không tìm thấy) là **105,011**.
- **500 Internal Server Error:** Số lượng lỗi 500 (Lỗi máy chủ nội bộ) là **14,266**.
- **502 Bad Gateway:** Số lượng lỗi 502 (Cổng chết) là **798**.
- **504 Gateway Timeout:** Số lượng lỗi 504 (Cổng thời gian chờ hết) là **103**.
- Các **mã trạng thái lỗi khác** như **499 Client Closed Request**, **400 Bad Request**, **403 Forbidden**, và **401 Unauthorized** cũng được liệt kê với số lần xuất hiện tương ứng.
- Dữ liệu **không hiển thị mã trạng thái** cho 2 yêu cầu, có thể là do dữ liệu bị thiếu hoặc bị lỗi trong quá trình ghi log. Trong một hệ thống thực tế, việc gặp phải các yêu cầu không có mã trạng thái có thể là do một số lỗi kỹ thuật hoặc vấn đề hệ thống. Điều này cần được kiểm tra và xử lý để đảm bảo tính toàn vẹn và đáng tin cậy của dữ liệu log.


In [ ]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("TotalRequestCount") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Đọc dữ liệu từ file logs vào RDD
log_rdd = spark.sparkContext.textFile(log_file_path)

# Đếm số lượng dòng trong RDD (tương ứng với số lượng yêu cầu)
total_request_count = log_rdd.count()

# Hiển thị tổng số lượng yêu cầu
print("Tổng số lượng yêu cầu:", total_request_count)

# Đóng SparkSession
spark.stop()

Tổng số lượng yêu cầu: 10365152


#Tính số lượng request theo giờ bằng map-reduce:

**Map**: Trích xuất giờ từ cột timestamp và gắn giá trị 1 cho mỗi giờ.

**Reduce**: Tổng hợp số lượng request cho mỗi giờ.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from operator import add

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("HourlyRequestAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào RDD
log_rdd = spark.sparkContext.textFile(log_file_path)

# Map: Trích xuất giờ từ timestamp và gắn giá trị 1 cho mỗi giờ
hourly_requests_rdd = log_rdd.map(lambda line: (line.split()[3].split(':')[1], 1))

# Reduce: Tổng hợp số lượng request cho mỗi giờ
hourly_request_counts = hourly_requests_rdd.reduceByKey(add)

# Hiển thị kết quả
hourly_request_counts_sorted = hourly_request_counts.sortByKey().collect()
for hour, count in hourly_request_counts_sorted:
    print(f"Hour {hour}: {count} requests")


Hour 00: 344800 requests
Hour 01: 227507 requests
Hour 02: 126406 requests
Hour 03: 79133 requests
Hour 04: 76481 requests
Hour 05: 70671 requests
Hour 06: 92867 requests
Hour 07: 184633 requests
Hour 08: 381551 requests
Hour 09: 563883 requests
Hour 10: 672849 requests
Hour 11: 731595 requests
Hour 12: 725359 requests
Hour 13: 724636 requests
Hour 14: 692415 requests
Hour 15: 649664 requests
Hour 16: 575790 requests
Hour 17: 548896 requests
Hour 18: 558690 requests
Hour 19: 566249 requests
Hour 20: 478688 requests
Hour 21: 415445 requests
Hour 22: 462359 requests
Hour 23: 414585 requests



**Dựa vào kết quả tính số lượng request theo giờ, chúng ta có thể nhận xét như sau:**

1. **Thời điểm cao điểm (Peak hours):** Các giờ cao điểm trong ngày là từ **9 giờ sáng đến 14 giờ chiều**, với số lượng request tăng đáng kể từ **563,883** đến **731,595** requests. Đây là thời điểm mà lượng truy cập vào hệ thống đạt đỉnh cao do nhu cầu sử dụng của người dùng tăng lên, có thể là do các hoạt động kinh doanh, mua sắm trực tuyến hoặc các dịch vụ trực tuyến khác đang hoạt động tích cực.

2. **Thời điểm thấp điểm (Off-peak hours)**: Các giờ thấp điểm trong ngày thường là từ **0 giờ đến 6 giờ sáng** và từ **20 giờ đến 23 giờ**, với số lượng request **giảm đáng kể xuống dưới 100,000 requests**. Đây có thể là thời gian mà người dùng thường ít hoặc không sử dụng dịch vụ, ví dụ như vào ban đêm khi họ đi ngủ hoặc vào buổi sáng sớm khi họ đang chuẩn bị cho một ngày mới.

3. **Mức độ tăng giảm ổn định:** Có sự ổn định trong mức độ tăng giảm của lượng truy cập trong suốt các giờ của ngày. Các biến động có thể phản ánh các mô hình hoạt động của người dùng hoặc các yếu tố khác như các chiến lược tiếp thị hoặc quảng cáo.

#Phân tích IP có số lượng truy cập cao:

Lọc ra các địa chỉ IP có số lần xuất hiện lớn hơn 10,000, coi chúng là địa chỉ IP nghi ngờ có lượng truy cập cao (high_traffic_ips)

Lọc ra các bản ghi có địa chỉ IP nằm trong danh sách high_traffic_ips.

Tính tổng thời gian truy cập cho mỗi địa chỉ IP trong mỗi giờ và hiển thị kết quả.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("NetworkAttackDetection") \
    .getOrCreate()

# Đường dẫn đến file log
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Đọc dữ liệu từ file log vào DataFrame
logs_df = spark.read.text(log_file_path)

# Phân tích log bằng cách áp dụng MapReduce
# Bước Map: Tạo cặp (địa chỉ IP, 1)
ip_counts_rdd = logs_df.rdd.map(lambda row: (row.value.split(" - - ")[0], 1))

# Bước Reduce: Tổng hợp số lần xuất hiện của mỗi địa chỉ IP
ip_counts = ip_counts_rdd.reduceByKey(lambda a, b: a + b)

# Lọc ra các địa chỉ IP nghi ngờ
suspicious_ips = ip_counts.filter(lambda x: x[1] > 10000)

# Hiển thị danh sách các địa chỉ IP nghi ngờ
suspicious_ips_df = spark.createDataFrame(suspicious_ips, ["ip_address", "count"])
suspicious_ips_df.show(truncate=False)

# Đóng SparkSession
spark.stop()


+---------------+------+
|ip_address     |count |
+---------------+------+
|104.222.32.94  |16706 |
|23.101.169.3   |27800 |
|5.160.157.20   |13526 |
|66.249.66.91   |314522|
|91.99.30.32    |45979 |
|5.117.116.238  |13427 |
|5.160.221.38   |12058 |
|17.58.102.43   |21698 |
|40.77.167.156  |10231 |
|195.181.168.164|24044 |
|91.99.47.57    |38612 |
|151.239.241.163|92475 |
|172.20.2.174   |11303 |
|66.249.66.194  |353483|
|91.99.72.15    |38694 |
|66.249.66.93   |22146 |
|66.249.66.92   |88332 |
|195.181.168.181|27979 |
|104.222.32.91  |42058 |
|5.78.190.233   |37204 |
+---------------+------+



**Dựa trên kết quả trên, chúng ta có thể nhận xét như sau:**

Có một số địa chỉ IP có số lượng truy cập rất lớn, ví dụ như IP 66.249.66.91 với 314,522 lần truy cập. Dựa vào số lượng lớn các yêu cầu từ địa chỉ IP 66.249.66.194 (353,483 yêu cầu) và địa chỉ IP 66.249.66.91 (314,522 yêu cầu), có thể kết luận rằng đây có thể là dấu hiệu của một loại tấn công mạng hoặc một hoạt động không đúng đắn từ phía người dùng hoặc bot. Điều này đặc biệt đáng chú ý vì số lượng yêu cầu từ địa chỉ IP này cao hơn đáng kể so với các địa chỉ IP khác trong danh sách. Cần kiểm tra kỹ lưỡng để đảm bảo rằng không có hoạt động bất thường hoặc không đúng đắn đang diễn ra từ địa chỉ IP này và có biện pháp phòng ngừa phù hợp nếu cần.

Có một số địa chỉ IP có số lần truy cập nhỏ hơn, nhưng vẫn đáng chú ý. Ví dụ, 104.222.32.94 chỉ có 16,706 lần truy cập, nhưng vẫn là một con số đáng kể.

Việc xác định nguyên nhân chính xác của số lượng truy cập cao hoặc thấp đối với mỗi địa chỉ IP cần phân tích thêm thông tin như loại request, thời gian truy cập, và các thông tin khác để đưa ra đánh giá chính xác.

#Phân tích thời gian truy cập của các địa chỉ IP

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from operator import add

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("HourlyAccessTimeAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào RDD
log_rdd = spark.sparkContext.textFile(log_file_path)

# Map: Trích xuất địa chỉ IP và thời gian truy cập từ timestamp
ip_access_time_rdd = log_rdd.map(lambda line: (line.split()[0], int(line.split()[3].split(':')[1])))

# Reduce: Tổng hợp thời gian truy cập cho mỗi địa chỉ IP trong mỗi giờ
ip_hourly_access_time = ip_access_time_rdd.reduceByKey(add)

# Hiển thị kết quả
ip_hourly_access_time_sorted = ip_hourly_access_time.sortByKey().collect()
for ip, hourly_access_time in ip_hourly_access_time_sorted:
    print(f"IP {ip}: Total access time per hour: {hourly_access_time}")


Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
IP 69.194.81.96: Total access time per hour: 0
IP 69.194.82.100: Total access time per hour: 20
IP 69.194.82.120: Total access time per hour: 420
IP 69.194.82.173: Total access time per hour: 1313
IP 69.194.82.2: Total access time per hour: 456
IP 69.194.82.216: Total access time per hour: 3
IP 69.194.82.67: Total access time per hour: 38
IP 69.194.82.79: Total access time per hour: 506
IP 69.194.82.85: Total access time per hour: 7
IP 69.194.82.90: Total access time per hour: 2758
IP 69.194.83.0: Total access time per hour: 390
IP 69.194.83.11: Total access time per hour: 600
IP 69.194.83.139: Total access time per hour: 260
IP 69.194.83.160: Total access time per hour: 252
IP 69.194.83.164: Total access time per hour: 50
IP 69.194.83.212: Total access time per hour: 60
IP 69.194.83.243: Total access time per hour: 976
IP 69.194.83.34: Total access time per hour: 32
IP 69.194.83.66: Total access time per hour: 55
IP 69.194.83.67

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
IP 93.126.3.186: Total access time per hour: 11
IP 93.126.31.59: Total access time per hour: 880
IP 93.126.34.182: Total access time per hour: 42
IP 93.126.34.98: Total access time per hour: 2170
IP 93.126.4.140: Total access time per hour: 2651
IP 93.126.40.18: Total access time per hour: 1650
IP 93.126.40.19: Total access time per hour: 1737
IP 93.126.40.4: Total access time per hour: 3690
IP 93.126.40.5: Total access time per hour: 1943
IP 93.126.40.6: Total access time per hour: 286
IP 93.126.44.164: Total access time per hour: 24
IP 93.126.44.53: Total access time per hour: 3352
IP 93.126.48.218: Total access time per hour: 45
IP 93.126.48.43: Total access time per hour: 23
IP 93.126.48.61: Total access time per hour: 17
IP 93.126.49.131: Total access time per hour: 189
IP 93.126.49.218: Total access time per hour: 1155
IP 93.126.49.237: Total access time per hour: 2160
IP 93.126.5.152: Total access time per hour: 28
IP 93.1

#Phân tích thời gian truy cập của các địa chỉ IP có số lượng truy cập cao

In [ ]:
from pyspark.sql import SparkSession
from operator import add

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("HourlyAccessTimeAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Danh sách các địa chỉ IP có số lượng truy cập cao
high_traffic_ips = ["104.222.32.94", "23.101.169.3", "5.160.157.20", "66.249.66.91", "91.99.30.32", "5.117.116.238",
                    "5.160.221.38", "17.58.102.43", "40.77.167.156", "195.181.168.164", "91.99.47.57",
                    "151.239.241.163", "172.20.2.174", "66.249.66.194", "91.99.72.15", "66.249.66.93",
                    "66.249.66.92", "195.181.168.181", "104.222.32.91", "5.78.190.233"]

# Đọc dữ liệu từ file logs vào RDD
log_rdd = spark.sparkContext.textFile(log_file_path)

# Map: Trích xuất địa chỉ IP và thời gian truy cập từ timestamp
ip_access_time_rdd = log_rdd.map(lambda line: (line.split()[0], int(line.split()[3].split(':')[1])))

# Lọc ra các bản ghi có địa chỉ IP thuộc danh sách high_traffic_ips
filtered_ip_access_time_rdd = ip_access_time_rdd.filter(lambda x: x[0] in high_traffic_ips)

# Reduce: Tổng hợp thời gian truy cập cho mỗi địa chỉ IP trong mỗi giờ
ip_hourly_access_time = filtered_ip_access_time_rdd.reduceByKey(add)

# Hiển thị kết quả
ip_hourly_access_time_sorted = ip_hourly_access_time.sortByKey().collect()
for ip, hourly_access_time in ip_hourly_access_time_sorted:
    print(f"IP {ip}: Total access time per hour: {hourly_access_time}")

# Đóng SparkSession
spark.stop()


IP 104.222.32.91: Total access time per hour: 649529
IP 104.222.32.94: Total access time per hour: 263359
IP 151.239.241.163: Total access time per hour: 1215741
IP 17.58.102.43: Total access time per hour: 232277
IP 172.20.2.174: Total access time per hour: 128907
IP 195.181.168.164: Total access time per hour: 297583
IP 195.181.168.181: Total access time per hour: 416867
IP 23.101.169.3: Total access time per hour: 319892
IP 40.77.167.156: Total access time per hour: 130421
IP 5.117.116.238: Total access time per hour: 186042
IP 5.160.157.20: Total access time per hour: 155740
IP 5.160.221.38: Total access time per hour: 163602
IP 5.78.190.233: Total access time per hour: 496687
IP 66.249.66.194: Total access time per hour: 4379765
IP 66.249.66.91: Total access time per hour: 3971303
IP 66.249.66.92: Total access time per hour: 1318346
IP 66.249.66.93: Total access time per hour: 348549
IP 91.99.30.32: Total access time per hour: 570382
IP 91.99.47.57: Total access time per hour: 504

 # Phân tích trường 'referrer'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from operator import add

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("LogAnalysisReferrer") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào DataFrame và phân tích các trường thông tin
parsed_logs_df = spark.read.text(log_file_path).select(
    regexp_extract('value', log_regex, 1).alias('ip_address'),
    regexp_extract('value', log_regex, 2).alias('timestamp'),
    regexp_extract('value', log_regex, 3).alias('request'),
    regexp_extract('value', log_regex, 4).alias('status_code'),
    regexp_extract('value', log_regex, 5).alias('response_size'),
    regexp_extract('value', log_regex, 6).alias('referrer'),
    regexp_extract('value', log_regex, 7).alias('user_agent'),
    regexp_extract('value', log_regex, 8).alias('additional_info')
)

# Phân tích trường 'referrer'
def analyze_referrer(log):
    referrer = log['referrer']
    return (referrer, 1)

referrer_counts = parsed_logs_df.rdd.map(analyze_referrer).reduceByKey(add).collect()

# Giới hạn số lượng bản ghi được in ra
LIMIT = 10

# Sắp xếp danh sách referrer_counts theo số lần xuất hiện giảm dần
sorted_referrers = sorted(referrer_counts, key=lambda x: x[1], reverse=True)

# Lặp qua danh sách referrer_counts và chỉ in ra giới hạn
for referrer, count in sorted_referrers[:LIMIT]:
    print(f"Referrer: {referrer}, Count: {count}")

# Đóng SparkSession
spark.stop()

Referrer: -, Count: 1646357
Referrer: https://www.zanbil.ir/, Count: 494757
Referrer: https://znbl.ir/static/bundle-bundle_site_head.css, Count: 387201
Referrer: https://www.zanbil.ir/browse/home-appliances/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C, Count: 219555
Referrer: https://www.zanbil.ir/m, Count: 128879
Referrer: https://www.google.com/, Count: 98420
Referrer: https://www.zanbil.ir/browse/cell-phone/%DA%AF%D9%88%D8%B4%DB%8C-%D9%85%D9%88%D8%A8%D8%A7%DB%8C%D9%84, Count: 87726
Referrer: https://www.zanbil.ir/m/browse/refrigerator-and-freezer/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1, Count: 79234
Referrer: https://www.zanbil.ir/browse/digital-supplies/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AF%DB%8C%D8%AC%DB%8C%D8%AA%D8%A7%D9%84, Count: 74759
Referrer: https://www.zanbil.ir/filter/b1,p62, Count: 59470


**Dựa vào số lượng truy cập từ mỗi Referrer, ta có thể đưa ra các nhận xét sau:**

1. Referrer '-' (không có thông tin): Số lượng truy cập lớn nhất đến từ Referrer không xác định, có thể là do người dùng truy cập trực tiếp vào trang web mà không thông qua một trang web trung gian.

2. Referrer 'https://www.zanbil.ir/' (Trang chính của Zanbil): Đây có thể là trang chính của Zanbil, nơi người dùng thường xuyên truy cập để tìm kiếm sản phẩm hoặc dịch vụ.

3. Referrer 'https://znbl.ir/static/bundle-bundle_site_head.css' (Tệp CSS của Zanbil): Có nhiều truy cập đến từ tệp CSS của Zanbil, có thể là do tệp CSS này được sử dụng trong nhiều trang của Zanbil.

4. Referrer 'https://www.zanbil.ir/browse/home-appliances/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AE%D8%A7%D9%86%DA%AF%DB%8C' (Danh mục sản phẩm 'Home Appliances' trên Zanbil): Người dùng thường truy cập vào danh mục sản phẩm 'Home Appliances' để tìm kiếm các sản phẩm liên quan đến đồ gia dụng.

5. Referrer 'https://www.zanbil.ir/m' (Phiên bản di động của Zanbil): Có một số truy cập đến từ phiên bản di động của trang web Zanbil.

6. Referrer 'https://www.google.com/' (Trang tìm kiếm Google): Một số người dùng đã truy cập Zanbil thông qua kết quả tìm kiếm trên Google.

7. Referrer 'https://www.zanbil.ir/browse/cell-phone/%DA%AF%D9%88%D8%B4%DB%8C-%D9%85%D9%88%D8%A8%D8%A7%DB%8C%D9%84' (Danh mục sản phẩm 'Cell Phones' trên Zanbil): Có truy cập đến danh mục sản phẩm 'Cell Phones', người dùng có thể đang tìm kiếm thông tin hoặc mua sản phẩm liên quan đến điện thoại di động.

8. Referrer 'https://www.zanbil.ir/m/browse/refrigerator-and-freezer/%DB%8C%D8%AE%DA%86%D8%A7%D9%84-%D9%81%D8%B1%DB%8C%D8%B2%D8%B1' (Danh mục sản phẩm 'Refrigerators and Freezers' trên Zanbil): Người dùng quan tâm đến danh mục sản phẩm 'Refrigerators and Freezers'.

9. Referrer 'https://www.zanbil.ir/browse/digital-supplies/%D9%84%D9%88%D8%A7%D8%B2%D9%85-%D8%AF%DB%8C%D8%AC%DB%8C%D8%AA%D8%A7%D9%84' (Danh mục sản phẩm 'Digital Supplies' trên Zanbil): Có truy cập đến danh mục sản phẩm 'Digital Supplies', người dùng có thể đang tìm kiếm các sản phẩm công nghệ số.

10. Referrer 'https://www.zanbil.ir/filter/b1,p62' (Bộ lọc sản phẩm trên Zanbil): Có một số truy cập đến từ việc sử dụng bộ lọc sản phẩm trên Zanbil.

**Dựa trên kết quả của Referrer**, ta có thể **tối ưu hóa chiến lược tiếp thị**, **cải thiện trải nghiệm người dùng**, **điều chỉnh chiến lược SEO và quảng cáo**, cũng như **theo dõi hiệu suất và đánh giá chiến lược một cách hiệu quả**.

#Phân tích trường 'user_agent'


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract
from operator import add

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("UserAgentAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào DataFrame và phân tích trường 'user_agent'
user_agent_df = spark.read.text(log_file_path).select(
    regexp_extract('value', log_regex, 7).alias('user_agent')
)

# Phân tích trường 'user_agent' để nhận biết trình duyệt phổ biến
def analyze_user_agent(user_agent):
    if 'Firefox' in user_agent:
        return ('Firefox', 1)
    elif 'Chrome' in user_agent:
        return ('Chrome', 1)
    elif 'Safari' in user_agent:
        return ('Safari', 1)
    else:
        return ('Other', 1)

# Áp dụng hàm phân tích trình duyệt lên DataFrame và thực hiện MapReduce
browser_counts = user_agent_df.rdd.map(lambda row: analyze_user_agent(row['user_agent'])).reduceByKey(add).collect()

# Hiển thị số lượng truy cập của mỗi trình duyệt
print("Số lượng truy cập của mỗi trình duyệt:")
for browser, count in browser_counts:
    print(f"{browser}: {count}")

# Phân tích trường 'user_agent' để hiểu hơn về hệ điều hành
def analyze_operating_system(user_agent):
    if 'Windows' in user_agent:
        return ('Windows', 1)
    elif 'Mac OS X' in user_agent:
        return ('Mac OS X', 1)
    elif 'Linux' in user_agent:
        return ('Linux', 1)
    else:
        return ('Other', 1)

# Áp dụng hàm phân tích hệ điều hành lên DataFrame và thực hiện MapReduce
os_counts = user_agent_df.rdd.map(lambda row: analyze_operating_system(row['user_agent'])).reduceByKey(add).collect()

# Hiển thị số lượng truy cập của mỗi hệ điều hành
print("\nSố lượng truy cập của mỗi hệ điều hành:")
for os, count in os_counts:
    print(f"{os}: {count}")

# Đóng SparkSession
spark.stop()


Số lượng truy cập của mỗi trình duyệt:
Chrome: 6136132
Safari: 862850
Other: 1099496
Firefox: 2266674

Số lượng truy cập của mỗi hệ điều hành:
Linux: 3633318
Windows: 5145246
Other: 925600
Mac OS X: 660988


**Dựa vào số liệu trên, chúng ta có thể nhận thấy một số điểm quan trọng về việc sử dụng trình duyệt và hệ điều hành:**

1. Trình duyệt phổ biến: Trình duyệt Chrome có số lượng truy cập cao nhất, gấp khoảng 6 lần so với Firefox và gần 7 lần so với Safari. Điều này cho thấy Chrome là trình duyệt phổ biến nhất trong tập dữ liệu này. Firefox cũng có một lượng truy cập đáng kể, nhưng vẫn thấp hơn so với Chrome. Safari và các trình duyệt khác có số lượng truy cập ít hơn.

2. Hệ điều hành phổ biến: Windows là hệ điều hành được sử dụng nhiều nhất, với số lượng truy cập cao nhất trong số các hệ điều hành. Tiếp theo là Linux và Mac OS X, tuy nhiên, số lượng truy cập của họ thấp hơn đáng kể so với Windows. Có một số truy cập được ghi nhận dưới dạng "Other", có thể là các hệ điều hành khác không được phân loại cụ thể.